In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [91]:
path = 'data/values_v2.csv'
values_df = pd.read_csv(path)

X_keys = ['tau_g', 'tau_gm']
Y_keys = ['faulty']
X_df = values_df[X_keys]
Y_df = values_df[Y_keys]
Y_df['neg_faulty'] = 1 - Y_df['faulty']
# Y_df

<ipython-input-91-8193e86714d8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_df['neg_faulty'] = 1 - Y_df['faulty']


In [61]:
class LSTMcovid(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LSTMcovid, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim)
        self.hidden2vals = nn.Linear(hidden_dim, 2)
    
    def forward(self, sequence):
        lstm_out, hidden = self.lstm(sequence)
        vals = self.hidden2vals(lstm_out)
        tag_scores = F.softmax(vals, dim=2)
        return tag_scores
    
    def suit4model(self, tsr):
        return tsr.unsqueeze(1).float()


class NN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 2)
    
    def forward(self, s):
        s = F.relu(self.fc1(s))
        s = F.relu(self.fc2(s))
        s = F.softmax(s, dim=2)
        return s
    
    def suit4model(self, tsr):
        return tsr.unsqueeze(1).float()
    

def calculate_acc(Y, Y_pred):
    Y_preds_args = np.argmax(Y_preds.detach().numpy(), axis=1)
    Y_args = np.argmax(Y.detach().numpy(), axis=1)
    res = Y_preds_args == Y_args
    return np.sum(res)/res.shape[0]
    

In [81]:
HIDDEN_DIM = 20
INPUT_DIM = len(X_keys)
model = LSTMcovid(INPUT_DIM, HIDDEN_DIM)
# model = NN(INPUT_DIM, HIDDEN_DIM)

In [92]:
X = torch.tensor(X_df.values).float()
X_norm = X / np.linalg.norm(X, axis=0)
Y = torch.tensor(Y_df.values).float()

In [96]:
with torch.no_grad():
    X_suited = model.suit4model(X_norm)
    Y_preds = model(X_suited)
    print(Y_preds, Y_preds.shape)
    print(Y, Y.shape)

tensor([[[0.5269, 0.4731]],

        [[0.5252, 0.4748]],

        [[0.5238, 0.4762]],

        ...,

        [[0.5198, 0.4802]],

        [[0.5198, 0.4802]],

        [[0.5198, 0.4802]]]) torch.Size([3551, 1, 2])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]) torch.Size([3551, 2])


In [107]:
# np.max(Y_preds.detach().numpy())
    
# print('acc', calculate_acc(Y, Y_preds.squeeze()))
print(Y_preds.squeeze())
Y_preds_args = np.argmax(Y_preds.squeeze().detach().numpy(), axis=1)
Y_args = np.argmax(Y.detach().numpy(), axis=1)
res = Y_preds_args == Y_args
np.sum(res)/res.shape[0]

tensor([[0.5269, 0.4731],
        [0.5252, 0.4748],
        [0.5238, 0.4762],
        ...,
        [0.5198, 0.4802],
        [0.5198, 0.4802],
        [0.5198, 0.4802]])


0.5167558434243875

In [108]:
EPOCHS = 20

loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(EPOCHS):
    print(f'epoch: {epoch}')
    model.zero_grad()

    suited_X = model.suit4model(X_norm)
    Y_preds = model(suited_X).squeeze()
    print(Y_preds, Y_preds.shape)
    print(Y, Y.shape)
#     print(Y_preds.type())
#     print(Y.type())
    
    print('acc', calculate_acc(Y, Y_preds))

    loss = loss_function(Y_preds, Y)
    loss.backward()
    optimizer.step()


epoch: 0
tensor([[0.5269, 0.4731],
        [0.5252, 0.4748],
        [0.5238, 0.4762],
        ...,
        [0.5198, 0.4802],
        [0.5198, 0.4802],
        [0.5198, 0.4802]], grad_fn=<SqueezeBackward0>) torch.Size([3551, 2])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]) torch.Size([3551, 2])
acc 0.5167558434243875
epoch: 1
tensor([[0.5269, 0.4731],
        [0.5252, 0.4748],
        [0.5238, 0.4762],
        ...,
        [0.5198, 0.4802],
        [0.5198, 0.4802],
        [0.5198, 0.4802]], grad_fn=<SqueezeBackward0>) torch.Size([3551, 2])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [1., 0.],
        [1., 0.],
        [1., 0.]]) torch.Size([3551, 2])
acc 0.5167558434243875
epoch: 2
tensor([[0.5269, 0.4731],
        [0.5252, 0.4748],
        [0.5238, 0.4762],
        ...,
        [0.5197, 0.4803],
        [0.5197, 0.4803],
        [0.5197, 0.4803]], grad_fn=<SqueezeBackward0>) t

In [50]:
Y_preds_args = np.argmax(Y_preds.detach().numpy(), axis=1)
Y_args = np.argmax(Y.detach().numpy(), axis=1)
res = Y_preds_args == Y_args
np.sum(res)/res.shape[0]

0.44959413459020686